<a href="https://colab.research.google.com/github/joaochenriques/Train/blob/main/Suspensao_V03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import *
from sympy.printing.lambdarepr import lambdarepr
init_printing() 
from IPython.display import Math
from scipy.integrate import odeint
import numpy as np

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

In [ ]:
import pathlib, subprocess
import matplotlib.pyplot as mpl

def cmdcall( cmd, verbose=True ):
    output = subprocess.getoutput( cmd )
    if verbose: print(output)

if not pathlib.Path("mpl_utils.py").exists():
    cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py" ) 

import mpl_utils as mut
mut.config_plots()

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

mpl.rcParams["figure.figsize"] = (6,4.5)

In [ ]:
t, ξ_0, ξ_1, ξ_2 = symbols( r"t, \xi_0, \xi_1, \xi_2" )
z_F1, z_F2, z_41, y_F1, y_F2 = symbols( r"z_{F1}, z_{F2}, z_{41}, y_{F1}, y_{F2}" )
h, h_z, ℓ_4, h_4, k_4, M, g, a_z, θ_0, I_θθ = symbols( r"h, h_z, \ell_4, h_4, k_4, M, g, a_z, \theta_0, I_{\theta\theta}" )
δ_y1, δ_z1, δ_y2, δ_z2, δ_ZM, δ_YM, θ = symbols( r"\delta_{y1}, \delta_{z1}, \delta_{y2}, \delta_{z2}, \delta_{ZM}, \delta_{YM}, \theta" )
F, ΔF, F_1, F_2, F_1y, F_1z, F_2y, F_2z, y_Fz, z_Fz = symbols( r"F, \Delta{F}, F_1, F_2, F_{1y}, F_{1z}, F_{2y}, F_{2z}, y_{Fz}, z_{Fz}" )
F_z = symbols( r"F_z" )
v_1, v_2, v_3 = symbols( r"v_1, v_2, v_3" )

In [ ]:
deg2rad = pi / 180

reps = { 
    ℓ_4: 0.7,
    h_4: 0.5,
    ξ_0: 0.5,
    k_4: 1E5,
    θ_0: 20*deg2rad,
    h: 1.2,
    h_z: 0.5,
    M: 13000/2,
    g: 9.8,
    a_z: 2,
    F_z: 1100,
    I_θθ: 14400,
    #
    ξ_1: sqrt( (z_F1 + ℓ_4)**2 + (y_F1-h_4)**2 ),
    ξ_2: sqrt( (z_F2 - ℓ_4)**2 + (y_F2 - h_4)**2 ),
    #
    F_1: k_4 * (ξ_0 - ξ_1) + M * g / ( 2 * cos(θ_0) ),
    F_2: k_4 * (ξ_0 - ξ_2) + M * g / ( 2 * cos(θ_0) ),
    F: (F_1 + F_2 ) / 2,
    ΔF: F_1 - F,
    #
    z_F1: δ_ZM + (h - (h_4+ξ_0*cos(θ_0) ) )*sin(θ) - ( ℓ_4 - ξ_0*sin(θ_0) )*cos(θ),
    z_F2: δ_ZM + (h - (h_4+ξ_0*cos(θ_0) ) )*sin(θ) + ( ℓ_4 - ξ_0*sin(θ_0) )*cos(θ),
    y_F1: h + δ_YM - (h - (h_4+ξ_0*cos(θ_0) ) )*cos(θ) - ( ℓ_4 - ξ_0*sin(θ_0) )*sin(θ),
    y_F2: h + δ_YM - (h - (h_4+ξ_0*cos(θ_0) ) )*cos(θ) + ( ℓ_4 - ξ_0*sin(θ_0) )*sin(θ),
    z_Fz: δ_ZM + (h - h_z )*sin(θ),
    y_Fz: h + δ_YM - (h - h_z )*cos(θ),
    #
    F_1z: F_1 * sin(θ_0 - θ),
    F_2z: F_2 * sin(θ_0 + θ),
    F_1y: F_1 * cos(θ_0 - θ),
    F_2y: F_2 * cos(θ_0 + θ),
    #
    δ_z1: δ_ZM - z_F1,
    δ_y1: h + δ_YM - y_F1,
    δ_z2: -δ_ZM + z_F2,
    δ_y2: h + δ_YM - y_F2
}
for key, values in reps.items():
    printmd( '$%s = %s$' % (latex(key), latex(values) ) )

In [ ]:
Exp1 = (F_1z - F_2z - F_z + M * a_z) / M
RHS1 = Exp1.subs( reps ).subs( reps ).subs( reps )
RHS1

In [ ]:
Exp2 = (F_1y + F_2y - M * g) / M 
RHS2 = Exp2.subs( reps ).subs( reps ).subs( reps )
RHS2

In [ ]:
Exp3 = -(F_1y*δ_z1 - F_1z*δ_y1 - F_2y*δ_z2 + F_2z*δ_y2 + F_z*(h - h_z)*cos(θ))/I_θθ 
RHS3 = Exp3.subs( reps ).subs( reps ).subs( reps )
RHS3

In [ ]:
U = [ v_1, v_2, v_3, δ_ZM, δ_YM, θ ]
func = lambdify( (U,t), [ RHS1, RHS2, RHS3, v_1, v_2, v_3 ] )
U0 = [0,0,0,0,0,0]
times = np.linspace(0, 10, 1001)

solution = odeint( func, U0, times ).T

In [ ]:
fig, ax1 = mpl.subplots()

mpl.plot( times, solution[3], label=r'$\delta_{ZM}$' )
mpl.plot( times, solution[4], label=r'$\delta_{YM}$' )
mpl.ylabel( r'$\delta$')
mpl.legend(loc='lower left');

ax2 = ax1.twinx() 
ax2.plot( times, solution[5]/deg2rad, 'g-', dashes = (5,1,1,1), label=r'$\theta$' )
ax2.set_ylabel( r'$\theta$' )
mpl.legend(loc='upper left');